In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

class LassoNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, lasso_coeff):
        super(LassoNet, self).__init__()
        
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
        
        self.lasso_coeff = lasso_coeff

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        output = self.linear2(x)
        return output

    def l1_penalty(self):
        l1_loss = 0
        for param in self.parameters():
            l1_loss += torch.sum(torch.abs(param))
        return self.lasso_coeff * l1_loss

In [3]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate synthetic data
X, y = make_regression(n_samples=1000, n_features=50, noise=0.1, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1))

# Instantiate LassoNet model
input_size = X_train.shape[1]
hidden_size = 20
output_size = 1
lasso_coeff = 0.01

model = LassoNet(input_size, hidden_size, output_size, lasso_coeff)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # L1 regularization term
    l1_loss = model.l1_penalty()
    
    # Total loss
    total_loss = loss + l1_loss

    # Backward pass and optimization
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Print the loss every few epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss.item():.4f}')

# Evaluate the model on the test set
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1))

with torch.no_grad():
    model.eval()
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)

print(f'Test Loss: {test_loss.item():.4f}')

Epoch [10/100], Loss: 21875.1934
Epoch [20/100], Loss: 18806.7207
Epoch [30/100], Loss: 15402.7646
Epoch [40/100], Loss: 18342.2637
Epoch [50/100], Loss: 20325.4414
Epoch [60/100], Loss: 21372.8301
Epoch [70/100], Loss: 20646.3652
Epoch [80/100], Loss: 27957.2773
Epoch [90/100], Loss: 14749.0908
Epoch [100/100], Loss: 28146.3867
Test Loss: 26613.2422
